<a href="https://colab.research.google.com/github/faisalisafk/python_scripts/blob/main/Named_entity_custom_with_SPACY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import spacy
from spacy import displacy
from wordcloud import WordCloud, STOPWORDS
from spacy.util import minibatch, compounding
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import re
import random
import math
import copy
from tqdm.notebook import tqdm
random.seed(42)

df = pd.read_csv('data.csv')

In [15]:
df.sample(10)

,transaction_descriptor,store_number,dataset
101,NNT BURLNGTON STORE472605,472605,validation
246,MCDONALD'S F33735,F33735,test
265,EXPRESS#0813,813,test
107,TACO BELL 729724,729724,validation
129,"WENDYS #514 TAMPA, FL (7271)",514,validation
194,BP#5998869CK ST,5998869,validation
286,STARBUCKS STORE 11966,11966,test
202,JCPENNEY 1419,1419,test
215,AEROPOSTALE # 864,864,test
56,MEIJER # 209 128,209,train


In [ ]:
df.count()

transaction_descriptor    300
store_number              300
dataset                   300
dtype: int64

In [17]:
train_df = df.loc[df['dataset'].isin(['train', 'validation'])]
test_df = df.loc[df['dataset'] == 'test']

print(train_df.shape)
test_df.shape

(200, 3)


(100, 3)

In [18]:
nlp0 = spacy.load('en_core_web_sm')

In [19]:
nlp0.pipe_names

['tagger', 'parser', 'ner']

In [20]:
ner0 = nlp0.get_pipe('ner')

In [21]:
def do_it(text,sub):
    start = text.find(sub)
    end = len(sub) + start
    entity = (start,end,'number')
    entities = []
    ent_dict = {}
    entities.append(entity)
    ent_dict['entities'] = entities
    #text = text.replace(" ", "0")
    train_item = (text, ent_dict)
    return train_item

In [22]:
TRAIN_DATA = []
TEST_DATA = []

for _, item in train_df.iterrows():
    train_item = do_it(item['transaction_descriptor'],item['store_number'])
    TRAIN_DATA.append(train_item)
print(len(TRAIN_DATA))

for _, item in test_df.iterrows():
    test_item = do_it(item['transaction_descriptor'],item['store_number'])
    TEST_DATA.append(test_item)
print(len(TEST_DATA))


200
100


In [23]:
n_iter = 50

def train_ner(training_data):
    """Steps
    Create a Blank NLP  model object
    Create and add NER to the NLP model
    Add Labels from your training data
    Train  
    """
    best = math.inf
    TRAIN_DATA = training_data
    nlp = spacy.blank("en")  # create blank Language class
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    with nlp.disable_pipes(*disable_pipes):
        optimizer = nlp.resume_training()

    nlp.begin_training()
    for itn in tqdm(range(n_iter)):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA)
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
                sgd=optimizer
            )
        print("Losses", losses)
        if losses['ner'] < best:
            best = losses['ner']
            best_nlp = copy.deepcopy(nlp)
            print('best changed')
    return best_nlp

In [24]:
%%time
nlp2 = train_ner(TRAIN_DATA)

  0%|          | 0/50 [00:00<?, ?it/s]

Losses {'ner': 329.6001978497952}
best changed
Losses {'ner': 124.40871342876926}
best changed
Losses {'ner': 42.934918562746795}
best changed
Losses {'ner': 34.99103732044751}
best changed
Losses {'ner': 10.175014131357829}
best changed
Losses {'ner': 5.6907935066856465}
best changed
Losses {'ner': 4.791624868742698}
best changed
Losses {'ner': 10.184309617981796}
Losses {'ner': 2.7336733734545127}
best changed
Losses {'ner': 3.9543982567469893}
Losses {'ner': 3.4535784476481686}
Losses {'ner': 0.019378743579666197}
best changed
Losses {'ner': 0.0014654791734449793}
best changed
Losses {'ner': 2.9019178543556703}
Losses {'ner': 0.12419026911953081}
Losses {'ner': 0.012667981784464813}
Losses {'ner': 1.6062353967577658}
Losses {'ner': 0.0003145087206554048}
best changed
Losses {'ner': 3.5942713140312796}
Losses {'ner': 0.017016569538959275}
Losses {'ner': 1.9997544675155814}
Losses {'ner': 3.8978177176920776}
Losses {'ner': 3.080881087518811}
Losses {'ner': 0.00035707457815120955}
Loss

In [25]:
#try random
temp = test_df.sample()
print(temp)
print("===============================")
text = temp['transaction_descriptor'].values[0]

docx2 = nlp2(text)

for entity in docx2.ents:
    print(entity)

displacy.render(nlp2(docx2.text),style='ent',jupyter=True)

    transaction_descriptor store_number dataset
298  BANANA REPUBLIC #8109         8109    test
8109


In [26]:
#try 223 cuz issues
temp = df.iloc[[223]]
print(temp)
print("===============================")
text = temp['transaction_descriptor'].values[0]

docx2 = nlp2(text)

for entity in docx2.ents:
    print(entity)

displacy.render(nlp2(docx2.text),style='ent',jupyter=True)

        transaction_descriptor store_number dataset
223  NNT POLO/RL WRENTHA130571        13057    test


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [29]:
#checking how many were good
count = 0
ok = []
for index, row in test_df.iterrows():

    docx2 = nlp2(row['transaction_descriptor'])
    try:   
        if str(docx2.ents[0]).lstrip('0') == str(row['store_number']):
            count+=1
            ok.append(index)   
    except:
        continue
print(count)
print(ok)

84
[200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 224, 225, 226, 227, 228, 229, 230, 232, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 266, 268, 269, 270, 271, 272, 273, 274, 275, 277, 278, 279, 280, 281, 284, 286, 287, 288, 289, 290, 294, 295, 296, 297, 298, 299]


In [30]:
#checking issues
count = 0
for index, row in test_df.iterrows():
    if index not in ok:
        docx2 = nlp2(row['transaction_descriptor'])
        try:
            result = str(docx2.ents[0])
        except:
            result = 'EMPTYYYYY'
        result = result.lstrip('0')

        print(str(row['transaction_descriptor']) + ' --- ' + str(row['store_number']) + ' --- ',result)
        count += 1

print(count)
 

BP#9442088LIBERTYVILLE B --- 9442088 ---  EMPTYYYYY
LBOUTLETS#4249 1475 N BUR --- 4249 ---  EMPTYYYYY
BP#8644346ES #30 B96 --- 8644346 ---  30
NNT POLO/RL WRENTHA130571 --- 13057 ---  EMPTYYYYY
NNT SEARS HOMETOWN 862751 --- 8627 ---  862751
NAVY EXCHANGE 050161 0003 --- 50161 ---  EMPTYYYYY
NNT FAMOUS FOOTWEAR001261 --- 1261 ---  EMPTYYYYY
EXPRESS#0813 --- 813 ---  EMPTYYYYY
SUNOCO 039962380 --- 39962380 ---  EMPTYYYYY
SUNOCO 0837208800        STATEN ISLANDNY --- 837208800 ---  EMPTYYYYY
NNT FAMOUS FOOTWEAR730376 --- 730376 ---  EMPTYYYYY
MARATHON PETRO170928   MIAMI --- 170928 ---  EMPTYYYYY
NNT FAMOUSFOOTWEAR#132427 --- 132427 ---  EMPTYYYYY
BP#1003300DANADA SQUS --- 1003300 ---  EMPTYYYYY
SUBWAY        00032128 --- 3212 ---  32128
TEXACO 00303733 --- 303733 ---  EMPTYYYYY
16
